## LearnR

Note that metapath names with `<` are [currently truncated](https://github.com/IRkernel/IRkernel/issues/286) in the notebook, unless they are specially HTML escaped.

In [1]:
library(dplyr, warn=F)

In [2]:
set.seed(0)

## Read datasets

In [10]:
dwpc_mat_df = readr::read_tsv('data/matrix/hetio-ind/features.tsv.bz2')

In [39]:
auroc_df = readr::read_tsv('data/auroc.tsv') %>%
  dplyr::filter(! grepl('PC', feature))
perm_affected = dplyr::filter(auroc_df, feature_type == 'degree' | fdr_pval_auroc <= 0.05)$feature
head(auroc_df, 2)

,feature_type,feature,nonzero,auroc,auroc_permuted,delta_auroc,pval_auroc,fdr_pval_auroc
1,degree,CbG,0.92238,0.67508,0.67481,0.00027521,0.92609,0.94922
2,DWPC,CbG<rG<rGaD,0.62464,0.78515,0.6887,0.096447,0.0017381,0.015367


In [40]:
length(perm_affected)

[1] 403

## Weighting

In [41]:
n_compounds = readr::read_tsv('../summary/compounds.tsv') %>% nrow()
n_diseases = readr::read_tsv('../summary/diseases.tsv') %>% nrow()
n_pairs = n_compounds * n_diseases
n_positives = sum(dwpc_mat_df$status == 1)
n_negatives = sum(dwpc_mat_df$status == 0)
c(n_pairs, n_positives, n_negatives)

[1] 209168    755   3020

In [42]:
positive_weight = 1
negative_weight = (n_pairs - n_positives) / n_negatives
weight_map = list('0' = negative_weight, '1' = positive_weight)
weight_map

$`0`
[1] 69.01093

$`1`
[1] 1

## Dataset preparation

In [43]:
head(dwpc_mat_df, 2)

,compound_id,compound_name,disease_id,disease_name,status,CbG,CbG<rG<rGaD,CbG<rG<rGdD,CbG<rG<rGuD,CbG<rGaD,ellip.h,CuGuDuGdD,CuGuDuGuD,DaG,DdG,DlA,DpC,DpS,DrD,DtC,DuG
1,DB01048,Abacavir,DOID:635,acquired immunodeficiency syndrome,1,3,4.95115e-05,0.000673428,0.000678888,0,⋯,0,0,40,250,25,0,24,2,14,250
2,DB01048,Abacavir,DOID:1459,hypothyroidism,0,3,9.89203e-05,0,0,0,⋯,0,0,44,0,24,0,34,3,4,0


In [44]:
features = auroc_df$feature
X_list = list()
X_list$all_features = X = dwpc_mat_df %>%
  dplyr::select(one_of(features)) %>%
  as.matrix()
X_list$perm_affected = dwpc_mat_df %>%
  dplyr::select(one_of(perm_affected)) %>%
  as.matrix()
y = dwpc_mat_df$status
w = as.numeric(weight_map[as.character(y)])
sprintf("%s compound–disease pairs × %s features", nrow(X), ncol(X))

[1] "3775 compound–disease pairs × 1213 features"

## Parameter Sweep

In [45]:
results = list()
i = 1
for (feature_set in names(X_list)) {
  for (seed in 1:10) {
    elem = list(seed = seed, feature_set = feature_set)
    elem$fit = hetior::glmnet_train(X = X_list[[feature_set]], y = y, alpha = 1, cores=12, seed=seed)
    elem$coef_df = elem$fit$coef_df %>%
      dplyr::filter(zcoef != 0) %>%
      dplyr::mutate(seed = seed, feature_set = feature_set)
    elem$pos_coefs = sum(elem$coef_df$zcoef > 0)
    elem$neg_coefs = sum(elem$coef_df$zcoef < 0)
    results[[i]] = elem
    i = i + 1
  }
}
length(results)

[1] 20

In [46]:
sweep_summary_df = do.call(rbind, lapply(results, function(x) {dplyr::data_frame(
    seed = x$seed,
    feature_set = x$feature_set,
    auroc = x$fit$vtm$auroc,
    pos_coefs = x$pos_coefs,
    neg_coefs = x$neg_coefs
)}))
sweep_summary_df

,seed,feature_set,auroc,pos_coefs,neg_coefs
1,1,all_features,0.9889935,30,19
2,2,all_features,0.9889935,30,19
3,3,all_features,0.9896912,32,24
4,4,all_features,0.9889935,30,19
5,5,all_features,0.9889935,30,19
6,6,all_features,0.9889935,30,19
7,7,all_features,0.9896912,32,24
8,8,all_features,0.9889935,30,19
9,9,all_features,0.990332,33,28
10,10,all_features,0.9889935,30,19


In [47]:
sweep_summary_df %>%
  dplyr::group_by(feature_set) %>%
  dplyr::summarize(
    mean = mean(auroc),
    sd = sd(auroc)
  )

,feature_set,mean,sd
1,all_features,0.9892669,0.0004735143
2,perm_affected,0.9877257,0.0007322548


In [48]:
p_all_features = subset(sweep_summary_df, feature_set == 'all_features')$auroc
p_perm_affected = subset(sweep_summary_df, feature_set == 'perm_affected')$auroc
t.test(p_all_features, p_perm_affected)


	Welch Two Sample t-test

data:  p_all_features and p_perm_affected
t = 5.5889, df = 15.407, p-value = 4.685e-05
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.000954749 0.002127572
sample estimates:
mean of x mean of y 
0.9892669 0.9877257 


In [49]:
sweep_coef_df = do.call(rbind, lapply(results, function(x) {x$coef_df}))
head(sweep_coef_df, 2)

,feature,coef,zcoef,seed,feature_set
1,CbGaD,23.70478,0.7320636,1,all_features
2,CbGaDdGuD,40.96539,0.1806922,1,all_features


In [50]:
sweep_feature_df = sweep_coef_df %>%
  dplyr::group_by(feature, feature_set) %>%
  dplyr::summarize(
    count = n()
  ) %>%
  dplyr::ungroup() %>%
  tidyr::spread(feature_set, count, fill=0) %>%
  dplyr::mutate(total = all_features + perm_affected) %>%
  dplyr::arrange(desc(all_features)) %>%
  dplyr::left_join(auroc_df)
head(sweep_feature_df, 2)

Joining by: "feature"


,feature,all_features,perm_affected,total,feature_type,nonzero,auroc,auroc_permuted,delta_auroc,pval_auroc,fdr_pval_auroc
1,CbGaD,10,10,20,DWPC,0.23311,0.75173,0.64228,0.10944,8.416e-06,0.001514
2,CbGaDdGuD,10,0,10,DWPC,0.27709,0.65788,0.6515,0.0063834,0.31739,0.42561


In [51]:
# Unique features
nrow(sweep_feature_df)

[1] 88

In [52]:
sweep_feature_df %>% readr::write_tsv('selection/sweep-features.tsv')
sweep_coef_df %>% readr::write_tsv('selection/sweep-coefficients.tsv')
sweep_summary_df %>% readr::write_tsv('selection/sweep-model-summaries.tsv')

## Fit single model

## Train model

Weights are currently not working due to an error thrown by `glmnet::cv.glmnet` (presumably [this line](https://github.com/cran/glmnet/blob/b8b39029eae71958e9c7c382240b7696fde3eff1/R/cv.lognet.R#L53)):

```
Error in predmat[which, seq(nlami)] = preds: replacement has length zero
```

Thus logistic regression model is fit without weights.

In [ ]:
fit = hetior::glmnet_train(X = X, y = y, alpha = 1, cores=12)

In [ ]:
# coef_df = fit$coef_df %>%
#   dplyr::filter(coef != 0) %>%
#   dplyr::left_join(auroc_df)

In [ ]:
# table(coef_df$zcoef %>% sign())

In [ ]:
# coef_df %>%
#   dplyr::mutate(feature = htmltools::htmlEscape(feature)) %>%
#   dplyr::arrange(zcoef)

In [ ]:
fit$vtm$auroc

In [ ]:
fit$vtm$auprc

In [ ]:
hetior::get_tjur(y_true = fit$y, y_pred = fit$y_pred)

In [ ]:
pred_df = dwpc_mat_df[1:7]
pred_df$prediction = fit$y_pred
head(pred_df)

In [ ]:
pred_df %>%
  ggplot2::ggplot(ggplot2::aes(prediction)) +
  ggplot2::geom_histogram(binwidth=0.01)

In [ ]:
pred_df %>%
  readr::write_tsv('data/predictions.tsv')

In [ ]:
tail(arrange(pred_df, prediction), 50)